In [ ]:
# https://realpython.com/primer-on-python-decorators/
# Decorator is...
    # a function that takes another function as argument
    # extends the behavior of that function, without explicitly modifying the function
    # decorates wrap a function, modifying its behavior

In [1]:
# ---------- First class Function ----------
# Functions are treated as objects / variables in Python
def square(x):
    return x * x

foo = square(5)     
print(square)       # <function square at 0x000001E00ED8DBD0>
print(foo)          # 25

# not square(), () means you're gonna execute the function
bar = square                # bar is a reference to the function square
print(bar)          # <function square at 0x000001E00ED8DBD0>
print(bar(5))       # 25

# ---------- Higher order Function, pass a func ----------
# Higher order function: a function that takes functions as arguments, and/or returns a function.
def my_map(func, arg_list):     # func is a function
    result = []
    for i in arg_list:
        result.append(func(i))
    return result

squares = my_map(square, [1, 2, 3, 4, 5])   # NOT square()
print(squares)       # [1, 4, 9, 16, 25]

# ---------- Higher order Function, return a func = Closure, see below ----------
def logger(msg):
    def log_message():
        print('Log:', msg)
    return log_message      # return a function

log_hi = logger('Hi!')    # log_hi is a reference to the function log_message with msg = 'Hi!'
log_hi()    # Log: Hi!

# Use case:     is this Command Pattern in Java?
def html_tag(tag):
    def wrap_text(msg):
        print('<{0}>{1}</{0}>'.format(tag, msg))
    return wrap_text
print_h1 = html_tag('h1')
print_h1('Test Headline!')      # <h1>Test Headline!</h1>
print_h1('Another Headline!')   # <h1>Another Headline!</h1>
print_p = html_tag('p')
print_p('Test Paragraph!')      # <p>Test Paragraph!</p>

<function square at 0x000001F4274E2050>
25
<function square at 0x000001F4274E2050>
25
[1, 4, 9, 16, 25]
Log: Hi!
<h1>Test Headline!</h1>
<h1>Another Headline!</h1>
<p>Test Paragraph!</p>


In [ ]:
# ---------- Closure ----------
# An inner function that remembers and has access to variables in the local scope in which it was created
# even after the outer function has finished executing

# Closure in Python. Criteria include 
# 1. must have a nested function
# 2. nested function must refer to a value defined in the enclosing outer function
# 3. enclosing function must return a reference of nested function
# consequence: inner function that remembers the variables in its enclosing scope
# even when the outer function is done executing, the variable goes out of scope
# or the function itself is removed from the current namespace. 

# Example 1
def outer_func(msg):
    message = msg    # message is a local variable, a 'free variable' to inner_func b/c it has access to it
    def inner_func():
        print(message)
    return inner_func    # NO (), so inner_func not executed, but return a function reference

hi_func = outer_func('Hi')    # my_func is a reference to the function object inner_func
# hi_func has access to both the 'message' variable and the inner_func function object (closure)
print(hi_func)              # <function outer_func.<locals>.inner_func at 0x000001E010F031C0>
print(hi_func.__name__)     # inner_func
# inner_func executed, has access to 'message' variable even after outer_func() is done
hi_func()                   # Hi
hello_func = outer_func('Hello')

# Example 2
def counter():
    count = 0
    def increment():
        nonlocal count
        count += 1
        return count
    return increment        # return reference to inner function

instance = counter()    # counter() function is removed from namespace after this line
                        # instance is now an function object reference that points to increment()
print(instance())       # but counter() function's inner function still works
print(instance())       # and it remembers count variable, i.e. it does not get refreshed to 0
print(instance())
# 1
# 2
# 3

# Example 3:
def make_multiplier_of(n):  # closure inner function remembers the value pass to n
    def multiplier(x):
        return x * n
    return multiplier

times3 = make_multiplier_of(3)  # Multiplier of 3    # these are decorator lines
times5 = make_multiplier_of(5)  # Multiplier of 5

print(times3(9))            # 27
print(times5(3))            # 15
print(times5(times3(2)))    # 30


<function outer_func.<locals>.inner_func at 0x000001E00ED8F370>
inner_func
Hi
Running "add" with arguments (3, 3)
6
Running "sub" with arguments (20, 5)
15


In [1]:
# ---------- Decorator basics ----------
# https://www.youtube.com/watch?v=FsAPt_9Bf3U

# Example 1: proof of concept
# this is boiler plate code for a decorator
def decorator(func):     # this is Decorator
    def wrapper(*args, **kwargs):   # *args, **kwargs: see decorator with arguments
        print("Do something before.")
        value = func(*args, **kwargs)
        print("Do something after.")
        return value 
    return wrapper

def say_whee():
    print("Whee!")

say_whee = decorator(say_whee)    # this line is omitted with @my_decorator
say_whee()
# Do something before.
# Whee!
# Do something after.
say_whee.__name__       # 'wrapper'  <= this is what happens when you don't use functools.wraps

@decorator
def say_whee_decorated():
    print("Decorated whee!")

say_whee_decorated()
# Do something before.
# Decorated whee!
# Do something after.

# Example2, the way the decorator modifies a function can change dynamically 
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # will not print at night time
    return wrapper

@not_during_the_night
def say_whee():
    print("Whee!")

say_whee()   # this will not print at from 22:00 to 7:00, otherwise - 'Whee!'

Do something before.
Whee!
Do something after.
Do something before.
Decorated whee!
Do something after.
Whee!


In [14]:
# ---------- Decorator with arguments / Decorator Factory / functools wraps ----------
# https://stackoverflow.com/questions/74207480/python-in-decorator-with-argument-how-are-arguments-passed-in/74207985#74207985 
# functools.wraps preserves the metadata of the original function

# Example 0: Structure
import functools

def decorator_factory(argument):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print('do stuff before')
            print('do sth with {argument}')
            result = func(*args, **kwargs)
            print('do stuff after')
            return result
        return wrapper
    return decorator

@decorator_factory('argument')
# this is the same as: say_whee = decorator_factory('argument')(foo), or
# decorator = decorator_factory('argument')
# foo = decorator(foo)
def foo(arg, kwarg):
    print(f'execute {arg} and {kwarg}')

foo('arg', 'kwarg')
# do stuff before
# do sth with {argument}
# execute arg and kwarg
# do stuff after

# Example 1: use case
import functools

def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)    # preserves the metadata of func, i.e. the original function name
        def wrapper(*args, **kwargs):   # *args, **kwargs: accepts func with arbitrary number arguments
            for _ in range(num_times):
                func(*args, *kwargs)
        return wrapper
    return decorator_repeat

# Method 1:
@repeat(num_times=2)    # decorator with arguments
def say_whee(name):     # you can pass a func with arguments to a decorator b/c of *args, **kwargs in wrapper
    print(f"Hello {name}")

say_whee('Alex')
# Hello Alex
# Hello Alex
say_whee        # <function __main__.say_whee(name)>
# without functools.wraps, <function __main__.repeat.<locals>.decorator_repeat.<locals>.wrapper(*args, **kwargs)>

# Method 2:
repeat_three_times = repeat(num_times=3)    # decorator factory

print(repeat_three_times.__closure__[0].cell_contents)      # 3

@repeat_three_times
def say_whee_three(name):
    print(f"Hello {name}")

say_whee_three('Three')
# Hello Three
# Hello Three
# Hello Three

# Method 3:
def say_whee_noAt(name):     # you can pass a func with arguments to a decorator b/c of *args, **kwargs in wrapper
    print(f"Hello {name}")

say_whee = repeat(num_times=4)(say_whee_noAt)    # this is the same as @repeat(num_times=4)
say_whee('Tom')
# Hello Tom
# Hello Tom
# Hello Tom
# Hello Tom

do stuff before
do sth with {argument}
execute arg and kwarg
do stuff after
Hello Alex
Hello Alex
dict_items([('__wrapped__', <function say_whee at 0x000001F9A4DDEDD0>)])
3
Hello Three
Hello Three
Hello Three
Hello Tom
Hello Tom
Hello Tom
Hello Tom


In [6]:
# ---------- Decorator method ----------

# Use case 1:
# track how many times a function is ran + the time it takes to run a function
# https://www.youtube.com/watch?v=FsAPt_9Bf3U
from functools import wraps

def my_logger(orig_func):
    import logging
    logging.basicConfig(filename=f'{orig_func.__name__}.log', level=logging.INFO)
    @wraps(orig_func)   # this line is needed to preserve the original function name
    def wrapper(*args, **kwargs):
        logging.info(f'Ran with args: {args}, and kwargs: {kwargs}')
        return orig_func(*args, **kwargs)
    return wrapper

def my_timer(orig_func):
    import time
    @wraps(orig_func)   # this line is needed to preserve the original function name
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print(f'{orig_func.__name__} ran in: {t2} sec')
        return result
    return wrapper

@my_timer
def display_time(param1, param2, param3):
    import time
    time.sleep(1)   # sleep for 1 sec
    print('display_time ran successfully')

display_time('whatever', 1, 2)
# display_time ran successfully
# display_time ran in: 1.003399133682251 sec

@my_logger
@my_timer
def display_info(name, age):
    import time
    time.sleep(1)   # sleep for 1 sec
    print(f'display_info ran with arguments ({name}, {age})')

# this is the same as   display_info = my_logger(my_timer(display_info))
display_info('Hank', 30)

Do something before.
Whee!
Do something after.
Do something before.
Decorated whee!
Do something after.
display_time ran successfully
display_time ran in: 1.0046217441558838 sec
display_info ran with arguments (Hank, 30)
display_info ran in: 1.0101714134216309 sec


In [4]:
# ---------- Decorator class ----------
# https://www.geeksforgeeks.org/class-as-decorator-in-python/
# Use decorator class if you want to keep track of the state of the function

# Example 0: explain __call__, this is not decorator
# __call__() special method allows class instances to behave like functions, i.e. can be called like a function
class Product:
    def __init__(self):
        print('Instance created')
    def __call__(self, a=5, b=10):
        print(a * b)
        
ans = Product() # Instance created
# __call__ method will be called here
ans(10, 20)     # 200
ans()           # 50

# Example 1: Structure
class decorator_class():
    def __init__(self, func):
        self.func = func      # tigh our function with an instance of the class
    def __call__(self, *args, **kwargs):
        print(f'call method executed before {self.func.__name__}')
        return self.func(*args, **kwargs)

@decorator_class
def display_info(name, age):
    print(f'display_info ran with arguments ({name}, {age})')

display_info('John', 25)
# call method executed before display_info
# display_info ran with arguments (John, 25)

# Example 2: Memorize state of the function
# class decorators act in the same way as a function decorator
# but a class can maintain and update a state
class CountCalls:
    def __init__(self, func):       #class decorator __init__ takes func
        self.func = func
        self.num_calls = 0          #keep tract of how many times func got executed
    
    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f'This is executed {self.num_calls} times')
        return self.func(*args, **kwargs)
    
@CountCalls
def say_hello():
    print('Hello')
    
say_hello()
# This is executed 1 times
# Hello
say_hello()
# This is executed 2 times
# Hello

call method executed before display_info
display_info ran with arguments (John, 25)
